In [114]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
import spacy
import torch
from torchtext import data
import torch.nn as nn
import re

In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
import re
import nltk
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import WordNetLemmatizer
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn, optim
import torchtext
from torchtext import data
import torch.nn.functional as F
from torchtext.vocab import Vectors
from sklearn.model_selection import train_test_split

In [156]:
spam=pd.read_csv(r"spam.csv", engine='python')

In [157]:
spam.head()

,text,category
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [158]:
spam = spam.drop_duplicates()
print('Shape after removing duplicates: ', spam.shape)
spam = spam.dropna()
print('Shape after removing null values: ', spam.shape)

Shape after removing duplicates:  (5169, 2)
Shape after removing null values:  (5169, 2)


In [161]:
spacy_en=spacy.load('en_core_web_sm')

stop_words=spacy.lang.en.STOP_WORDS
PUNCT_TO_REMOVE = string.punctuation
lemmatizer = WordNetLemmatizer()

# First checking if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ', device)

Device:  cuda


In [162]:
spam['text'] = spam['text'].str.lower()

# removing punctuation marks
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
spam["text"] = spam["text"].apply(lambda text: remove_punctuation(text))


# Removing stopwords
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOP_WORDS])
spam["text"] = spam["text"].apply(lambda text: remove_stopwords(text))

        
# Lemmatizing the text data
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
spam["text"] = spam["text"].apply(lambda text: lemmatize_words(text))

# to clean data
def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

spam["text"] = normalise_text(spam["text"])

In [182]:

spam = spam.drop_duplicates()
spam = spam[spam['text'] != ""]


In [164]:
train_df, valid_df = train_test_split(spam)
train_df.to_csv('train_spam1.csv', index=False)
valid_df.to_csv('test_spam1.csv', index=False)

In [175]:
spacy_en = spacy.load('en_core_web_sm')

def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [167]:

import torchtext
from torchtext.data import Field

In [176]:
TEXT =data.Field(sequential=True, tokenize=tokenizer, lower=True)
LABEL = Field(sequential=False)

#clean the text


In [177]:
train_datafield = [('text', TEXT),  ('label', LABEL)]
train = TabularDataset(path ='train.csv',  
                             format='csv',
                             skip_header=True,
                             fields=train_datafield)


#%%
test_datafield = [('text', TEXT),  ('label',LABEL)]

test = TabularDataset(path ='valid.csv', 
                       format='csv',
                       skip_header=True,
                       fields=test_datafield)

In [178]:
print(train[0].text,  train[0].label)
print(test[0].text,  test[0].label)

['so', 'what', 'u', 'doing', 'today', '?'] ham
['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...'] ham


In [179]:
TEXT.build_vocab(train, vectors= 'glove.6B.100d')
LABEL.build_vocab(train)
#%% load the pretrained embedding
vocab = TEXT.vocab


train_iter = Iterator(
        train, 
        batch_size=64,
        device=torch.device('cuda'), 
        sort_within_batch=False,
        repeat=False)

test_iter = Iterator(test, batch_size=32, device=torch.device('cuda'), 
                     sort_within_batch=False, repeat=False)

In [180]:
class textCNN(nn.Module):
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        super(textCNN, self).__init__()
        #load pretrained embedding in embedding layer.
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)
    
        #Convolutional Layers with different window size kernels
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        #Dropout layer
        self.dropout = nn.Dropout(0.6)
        
        #FC layer
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)
        
    def forward(self, x):
        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)

        con_x = [conv(emb_x) for conv in self.convs]

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]
        
        fc_x = torch.cat(pool_x, dim=1)
        
        fc_x = fc_x.squeeze(-1)

        fc_x = self.dropout(fc_x)
        logit = self.fc(fc_x)
        return logit
        

#%% Training the Model
def train(model, device, train_itr, optimizer, epoch, max_epoch):
    model.train()
    corrects, train_loss = 0.0,0
    for batch in train_itr:
        text, target = batch.text, batch.label
        text = torch.transpose(text,0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        optimizer.zero_grad()
        logit = model(text)
        
        loss = F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
    
    size = len(train_itr.dataset)
    train_loss /= size 
    accuracy = 100.0 * corrects/size
  
    return train_loss, accuracy
    
def valid(model, device, test_itr):
    model.eval()
    corrects, test_loss = 0.0,0
    for batch in test_itr:
        text, target = batch.text, batch.label
        text = torch.transpose(text,0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)

        
        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
    
    size = len(test_itr.dataset)
    test_loss /= size 
    accuracy = 100.0 * corrects/size
    
    return test_loss, accuracy

In [181]:
model = textCNN(vocab, 100, 100, [3, 4 , 5] , 2).to('cuda')
# print the model summery
print(model)    
    
train_loss = []
train_acc = []
test_loss = []
test_acc = []
best_test_acc = -1

# Use GPU if it is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

#optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 10+1):
    #train loss
    tr_loss, tr_acc = train(model, device, train_iter, optimizer, epoch, 100)
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    ts_loss, ts_acc = valid(model, device, test_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, ts_loss, ts_acc))
    
    if ts_acc > best_test_acc:
        best_test_acc = ts_acc
        #save paras(snapshot)
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "textCNN_IMDB_best_valid")
        
    train_loss.append(tr_loss)
    train_acc.append(tr_acc)
    test_loss.append(ts_loss)
    test_acc.append(ts_acc)


textCNN(
  (embed): Embedding(7816, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.6, inplace=False)
  (fc): Linear(in_features=300, out_features=2, bias=True)
)
Train Epoch: 1 	 Loss: 0.003690009249899632 	 Accuracy: 91.66666412353516%
Valid Epoch: 1 	 Loss: 0.0034275024886461394 	 Accuracy: 95.93301391601562%
model saves at 95.93301391601562 accuracy
Train Epoch: 2 	 Loss: 0.0011505308713859472 	 Accuracy: 97.6153793334961%
Valid Epoch: 2 	 Loss: 0.0022127470848235216 	 Accuracy: 97.96650695800781%
model saves at 97.96650695800781 accuracy
Train Epoch: 3 	 Loss: 0.0006175840132607099 	 Accuracy: 98.82051086425781%
Valid Epoch: 3 	 Loss: 0.0018409502953758829 	 Accuracy: 98.20573425292969%
model saves at 98.20573425292969 accuracy
Train Epoch: 4 	 Loss: 0.00037075113421544816 	 Accuracy: 99.4358